In [ ]:
from google.colab import drive
drive.mount('/content/mnt')

# Notebook Set-Up

## Imports

In [ ]:
workingDirectory = 'mnt/My Drive/AI Project/CRM Convergence/'

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from google.colab import drive

import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics
from keras import backend as K

import os
import datetime
import random

!cp 'mnt/My Drive/AI Project/CRM Convergence/rbflayer.py' .
!cp 'mnt/My Drive/AI Project/CRM Convergence/kmeans_initializer.py' .
from rbflayer import RBFLayer
from kmeans_initializer import InitCentersKMeans

Using TensorFlow backend.


In [ ]:
def csv_to_np_array(file_path):
  return np.array(pd.read_csv(file_path))

def shuffle_split(a, seed):
  np.random.seed(seed)
  np.random.shuffle(a)
  temp = np.split(a, indices_or_sections=[0, int(0.2*len(a)+1)])
  return temp[1], temp[2]

## TensorBoard

In [ ]:
%load_ext tensorboard

# Regression

## Data Preprocessing

In [ ]:
x = csv_to_np_array(workingDirectory + 'parameters.csv')
y = csv_to_np_array(workingDirectory + 'solver_settings.csv')

In [ ]:
#normx = preprocessing.MinMaxScaler()
#normy = preprocessing.MinMaxScaler()
#x = normx.fit_transform(x)
#y = normy.fit_transform(y)

In [ ]:
seed = random.randrange(0,99999)
#3192861
#11737

xtest, xtrain = shuffle_split(x, seed)
ytest, ytrain = shuffle_split(y, seed)

## Model

In [ ]:
rbflayer = RBFLayer(10, initializer=InitCentersKMeans(xtrain), betas=3.0, input_shape=(2,))

model = tf.keras.models.Sequential()
model.add(rbflayer)
model.add(tf.keras.layers.Dense(5, activation = 'linear', name = 'output'))

#model.add(tf.keras.layers.Dense(100, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))

tb_cb = tf.keras.callbacks.TensorBoard(log_dir="logs/" + datetime.datetime.now().strftime("%H:%M"), histogram_freq=1)

## Training

In [ ]:
tf.keras.optimizers.Adam(learning_rate=0.5)

In [ ]:
model.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics = 'mean_absolute_error')

In [ ]:
model.fit(x=xtrain,y=ytrain, epochs = 4000, verbose=2, callbacks=[tb_cb], initial_epoch= 0, validation_data=(xtest, ytest))

# Results

In [ ]:
%tensorboard --logdir logs/

In [ ]:
model.save(workingDirectory+'model.h5')

In [ ]:
model.predict(xtest)

array([[73.04498  ,  2.0204234, 63.15815  ,  4.9163938, 18.117094 ],
       [69.640785 ,  2.0584462, 58.95158  ,  4.5767846, 18.37493  ],
       [70.08429  ,  2.0199301, 59.89814  ,  4.7672887, 18.436012 ],
       [69.747284 ,  2.0476851, 59.30294  ,  4.953621 , 18.426012 ],
       [70.44469  ,  2.0299363, 60.268322 ,  4.9140797, 18.196537 ],
       [68.78324  ,  2.0288596, 58.094276 ,  4.5201054, 18.762434 ],
       [74.18298  ,  2.0135572, 64.41036  ,  4.823111 , 18.276743 ],
       [77.97125  ,  2.0424886, 68.113106 ,  4.9666467, 18.670265 ],
       [68.799324 ,  2.0832553, 57.750755 ,  4.8800254, 18.914408 ],
       [69.216995 ,  2.0892613, 58.109283 ,  4.9640746, 19.15608  ],
       [87.559555 ,  2.114216 , 77.39436  ,  4.983566 , 20.416954 ],
       [69.29065  ,  2.0975232, 58.18598  ,  5.0613294, 19.05013  ],
       [69.454384 ,  2.0395682, 58.83497  ,  4.4916644, 18.45035  ],
       [69.295876 ,  2.0882   , 58.22352  ,  4.981151 , 19.135527 ],
       [73.81801  ,  2.0697954, 62

In [ ]:
ytest

array([[69.99277517,  2.        , 78.        ,  6.        , 14.        ],
       [78.35581069,  3.        , 29.        ,  3.        , 31.        ],
       [66.20494347,  2.        , 63.        ,  5.        , 28.        ],
       [67.4830485 ,  2.        , 68.        ,  2.        , 29.        ],
       [64.00241472,  2.        , 56.        ,  8.        , 25.        ],
       [81.97591771,  3.        , 29.        ,  8.        , 27.        ],
       [77.07153652,  3.        , 45.        ,  3.        , 25.        ],
       [55.99365029,  2.        , 97.        ,  4.        , 18.        ],
       [52.1915459 ,  3.        , 43.        ,  7.        , 31.        ],
       [83.71302203,  2.        , 43.        ,  5.        , 20.        ],
       [72.84756995,  3.        , 85.        ,  3.        , 22.        ],
       [59.84141901,  3.        , 36.        ,  3.        , 20.        ],
       [55.13647351,  2.        , 57.        ,  2.        , 14.        ],
       [99.40258666,  2.        , 80. 

In [ ]:
#inversed
invtest = csv_to_np_array(workingDirectory + 'solver_settings.csv')[0:18]
ypred = model.predict(xtest)
#invpred = normy.inverse_transform(model.predict(xtest))
#invmse = np.array([metrics.mean_absolute_error(invtest[:,0], invpred[:,0]), metrics.mean_absolute_error(invtest[:,1], invpred[:,1]), metrics.mean_absolute_error(invtest[:,2], invpred[:,2]), metrics.mean_absolute_error(invtest[:,3], invpred[:,3]),metrics.mean_absolute_error(invtest[:,4], invpred[:,4])])
mae = np.array([metrics.mean_absolute_error(invtest[:,0], ypred[:,0]), metrics.mean_absolute_error(invtest[:,1], ypred[:,1]), metrics.mean_absolute_error(invtest[:,2], ypred[:,2]), metrics.mean_absolute_error(invtest[:,3], ypred[:,3]),metrics.mean_absolute_error(invtest[:,4], ypred[:,4])])
mae

array([12.8414203 ,  0.39896664, 14.67004861,  1.43496082,  5.72468482])

In [ ]:
print(np.average(mse))

7.014016238294644


In [ ]:
print(metrics.mean_squared_error(invtest, ypred))

118.94085392161602


In [ ]:
print(xtrain)

[[7.5694479  0.79102083]
 [8.17839884 0.89913943]
 [8.84002532 0.81412505]
 [9.83787806 0.75386283]
 [8.30431404 0.78348225]
 [9.22483212 0.82776831]
 [9.52043916 0.79603691]
 [9.61365464 0.79494281]
 [7.64618948 0.83360887]
 [7.27849925 0.75247925]
 [7.10163553 0.86236306]
 [9.36335747 0.84881386]
 [8.55120909 0.89094977]
 [7.13145234 0.7766419 ]
 [9.45827704 0.88997581]
 [7.35940584 0.77117134]
 [8.32516308 0.79304843]
 [9.90223407 0.78598636]
 [7.67226556 0.77815292]
 [8.962688   0.89314093]
 [9.02308548 0.89549065]
 [8.41169425 0.80884223]
 [7.08315416 0.76045051]
 [8.21982782 0.7915149 ]
 [8.98297422 0.78135288]
 [8.53373213 0.82381804]
 [8.80953403 0.76449425]
 [7.18740602 0.8436702 ]
 [7.79596046 0.81726148]
 [8.39992348 0.77978142]
 [8.65765572 0.85308542]
 [7.89819445 0.77342307]
 [8.72238583 0.86718853]
 [7.44864352 0.75098367]
 [7.51885005 0.75689047]
 [8.69404346 0.8022818 ]
 [7.21114179 0.81178865]
 [7.68920629 0.78764335]
 [9.91874312 0.80126254]
 [7.02275335 0.89575819]


In [ ]:
print(xtest)
print(invtest)
print(ypred)

[[7.58359267 0.89185824]
 [9.40985386 0.86634402]
 [7.92364636 0.81564506]
 [8.00487206 0.87068615]
 [7.84393329 0.88086835]
 [9.13561955 0.76888274]
 [7.51081496 0.8572685 ]
 [7.33492784 0.86870134]
 [8.89619719 0.83728386]
 [8.59799691 0.81652805]
 [7.05694569 0.77505433]
 [8.46704272 0.85208775]
 [9.3819769  0.82974551]
 [8.50517115 0.81935019]
 [9.7473128  0.83972069]
 [7.86720716 0.87807211]
 [8.23088858 0.76847893]
 [8.76373503 0.75458917]]
[[54.30585644  2.         44.          4.         17.        ]
 [85.45391183  2.         58.          7.         15.        ]
 [85.92549545  3.         78.          5.         15.        ]
 [59.35286581  3.         97.          7.         32.        ]
 [86.22787686  2.         53.          7.         12.        ]
 [97.10400007  2.         50.          7.         11.        ]
 [71.84974059  2.         65.          6.         13.        ]
 [55.13647351  2.         57.          2.         14.        ]
 [52.84546113  2.         82.          4.    

In [ ]:
examples = [[7.0, 0.87], [8.0, 0.95], [5.0, 0.5]]
print(model.predict(examples))

[[ 89.938286    2.171447   79.31018     5.4922833  20.536835 ]
 [ 69.852905    2.0710726  59.259636    5.1661696  18.256136 ]
 [646.93713    10.307213  585.9045     25.519045  123.943634 ]]
